In [ ]:
import sys
sys.path.append('..')
from pcutils.core.track_generator import TrackInterpolator, InitialState
from pcutils.core.config import load_config
from pcutils.core.workspace import Workspace
from pcutils.core.particle_id import load_particle_id
from pcutils.core.target import Target
from pcutils.core.solver_interp import fit_model, Guess, generate_trajectory
from pcutils.core.cluster import Cluster

import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import h5py as h5
import numpy as np


In [ ]:
config = load_config('../local_config.json')
ws = Workspace(config.workspace)
track_path = ws.get_track_file(config.solver.interp_file_name)
tracks = TrackInterpolator(track_path)
nuc_map = ws.get_nuclear_map()
pid = load_particle_id(ws.get_gate_file_path(config.solver.particle_id_filename), nuc_map)
target = Target(config.solver.gas_data_path, nuc_map)

In [ ]:
run_number = 4
cluster_file = h5.File(ws.get_cluster_file_path(run_number))
estimate_df = pl.scan_parquet(ws.get_estimate_file_path_parquet(run_number))
estimate_gated = estimate_df.filter(pl.struct(['dEdx', 'brho']).map(pid.cut.is_cols_inside)).collect().to_dict()
cluster_group = cluster_file['cluster']
nrows = len(estimate_gated['event'])
row = np.random.randint(0, nrows)
# row = 2413
print(f'row: {row}')
event = estimate_gated['event'][row]
cluster_index = estimate_gated['cluster_index'][row]
print(f'event: {event}')
print(f'cluster index: {cluster_index}')
event_group = cluster_group[f'event_{event}']
local_cluster = event_group[f'cluster_{cluster_index}']
print(f'Direction: {estimate_gated["direction"][row]}')
cluster = Cluster(event, local_cluster.attrs['label'], local_cluster['cloud'][:].copy())
cluster.z_bin_width = local_cluster.attrs['z_bin_width']
cluster.z_bin_low_edge = local_cluster.attrs['z_bin_low_edge']
cluster.z_bin_hi_edge = local_cluster.attrs['z_bin_hi_edge']
cluster.n_z_bins = local_cluster.attrs['n_z_bins']

In [ ]:
guess = Guess(estimate_gated['brho'][row], estimate_gated['polar'][row], estimate_gated['azimuthal'][row], estimate_gated['vertex_x'][row], estimate_gated['vertex_y'][row], estimate_gated['vertex_z'][row])
result = fit_model(cluster, guess, tracks, pid.nucleus)
if result is None:
    print('Guess outside of interpolation range!')
best_fit_trajectory_xy = generate_trajectory(result, tracks, pid.nucleus)(cluster.data[:, 2]*0.001)
cluster.data[:, :3] *= 0.001

In [ ]:
plt.scatter(cluster.data[:, 0], cluster.data[:, 1], s=3, label='data')
plt.scatter(best_fit_trajectory_xy[:, 0], best_fit_trajectory_xy[:, 1], s=3, label='fit')
plt.legend()


In [ ]:
plt.scatter(cluster.data[:, 2], cluster.data[:, 1], s=3, label='data')
plt.scatter(cluster.data[:, 2], best_fit_trajectory_xy[:, 1], s=3, label='fit')


In [ ]:
plt.scatter(cluster.data[:, 2], cluster.data[:, 0], s=3, label='data')
plt.scatter(cluster.data[:, 2], best_fit_trajectory_xy[:, 0], s=3, label='fit')